In [10]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import  re


print(u'正在连接chrome浏览器...')
driver = webdriver.Chrome('/Users/Hebbelu/Downloads/chromedriver')
print(u'已打开chrome浏览器，并成功连接')

正在连接chrome浏览器...


已打开chrome浏览器，并成功连接


In [15]:
#载入帐号信息，参考模版user_data_temp.json

import json

def load_data():
    with open('/Users/Hebbelu/PycharmProjects/otcbtc/'
              'luyh.json') as f:  
        data = json.load(f)
        #print(data)
    #print(data["user"],data["password"])
    return data

data = load_data()

In [16]:
class otcbtc(object):
    def __init__(self,driver):
        self.dr = driver
        
    def login(self):
        print(u'正在打开otcbtc.com')
        self.dr.get( 'http://www.otcbtc.com' )
        print(u'已打开otcbtc.com')
        time.sleep( 1 )
        
        print(u'正在使用帐号密码登陆...')
        buttun = self.dr.find_element_by_class_name( 'log-in-btn' )
        buttun.click()
        time.sleep( 1 )

        user = self.dr.find_element_by_id( 'user_email' )
        user.send_keys( data["user"] )

        pwd = self.dr.find_element_by_id( 'user_password' )
        pwd.send_keys( data["password"] )

        log_in_btn = self.dr.find_element_by_name( 'commit' )
        log_in_btn.click()
        time.sleep( 1 )
        print(u'已帐号密码登陆otcbtc.com')

        print( u"请输入验证码" )
        time.sleep( 5 )
        
        self._check_login()

    def get_market_otceth(self):
        print( u"正在切换至otbeth币币交易" )
        self.dr.get("https://bb.otcbtc.com/exchange/markets/otbeth")
        print( u"已切换至otbeth币币交易" )
        time.sleep(5)

    def _check_login(self):
        time.sleep(5)
            #TODO:先用将就延迟代替，再完善
    
    def _get_current_url(self):
        return self.dr.current_url
 
otc = otcbtc(driver)
otc.login()
time.sleep(5)




正在打开otcbtc.com


已打开otcbtc.com


正在使用帐号密码登陆...


已帐号密码登陆otcbtc.com
请输入验证码


In [17]:
# 登陆otc_eth币币交易

otc.get_market_otceth()

正在切换至otbeth币币交易


已切换至otbeth币币交易


In [64]:

class  trade(object):    
    def __init__(self,driver):
        self.dr = driver
        # self.buy1price = None
        # self.sell1price = None
        # self.cjbfb = None
        # self.money = None
        # self.bi = None
        # self.buy_order_price = None
        # self.buy_amount = None
 
    def _get_buy1_price(self):
        buy = self.dr.find_element_by_xpath("//div[@class='bid-book']//ul[2]/li/span").text
        self.buy1price =  float(buy)
            
    def _get_sell1price(self):
        sell = self.dr.find_element_by_xpath("//div[@class='ask-book']//ul[2]/li/span").text
        self.sell1price = float(sell)
   
    def _chajia(self):
        cjbfb = self.sell1price /self.buy1price -1
        self.cjbfb = cjbfb*100
        
    def update_price(self,show = 0):
        self._get_buy1_price()
        self._get_sell1price()
        self._chajia()
        if show:
            print(u'买一价',round(self.buy1price,8))
            print(u'卖一价',round(self.sell1price,8))
            print(u'现有差价',round(self.cjbfb,2),'%')
            
    def get_money(self):    
        xpath = "/html/body/div[2]/div/div/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/" \
            "ul/li[2]/span[2]"
        element = self.dr.find_element_by_xpath(xpath)
        try:
            text = re.findall('(\d+(\.？\d+))',element.text)[0]
        except:
            text = 0
        self.money = float(text)
        
    def get_bi(self):
        xpath = "/html/body/div[2]/div/div/div/div[2]/div[2]/div[2]/div/div[2]/div[2]/ul/li[2]/span[2]"
        element = self.dr.find_element_by_xpath(xpath)
        text = re.findall('\d+\.?\d+',element.text)[0]
        self.bi = float(text)
        
    def _get_buy_order_price(self):
        self._get_buy1_price()
        self.buy_order_price = round(self.buy1price+0.00000001,8)
      
    def _input_buy_price(self):
        self._get_buy_order_price()
        
        xpath  = "/html/body/div[2]/div/div/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/form/div[1]/div[1]/input"
        buy_input = self.dr.find_element_by_xpath(xpath)
        buy_input.clear()
        buy_input.send_keys(str(self.buy_order_price))

    def _get_buy_amount(self):
        self.buy_amount = round(self.money/self.buy_order_price,6)-0.000001
        
    def _input_buy_amount(self):
        self._get_buy_amount()
        
        xpath = "/html/body/div[2]/div/div/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/form/div[2]" \
            "/div/input"
        amount_input = self.dr.find_element_by_xpath(xpath)
        amount_input.clear()
        amount_input.send_keys(str(self.buy_amount))
        
    def _commit_buy_order(self):
        commit = self.dr.find_element_by_xpath("/html/body/div[2]/div/div/div/div[2]/div[2]/div[2]/div/div[1]/div[2]/form/div[4]/button")
        print(commit.text)
        commit.send_keys(Keys.ENTER)
    
    def buy_order(self):
        self._input_buy_price()
        self._input_buy_amount()
        self._commit_buy_order()
         
    def cancle_buy_order(self):
        xpath = "//ul[@class='general-flex-list scroll-block history-lists']"
        cancle = self.dr.find_element_by_xpath(xpath+
                "/li//span[7]/a")        
        #print(cancle.text)
        cancle.send_keys(Keys.ENTER)  
        time.sleep(5)
        
    def get_my_orders(self):
        xpath ="/html/body/div[2]/div/div/div/div[2]/div[3]/div[2]/div[1]/div[2]/ul[2]"
        orders_text = self.dr.find_element_by_xpath(xpath).text
        orders = re.split('\n',orders_text)
        #print(orders)
        return orders
    
    def check_buy_order(self):
        self._get_buy1_price()
        if self.buy1price > self.buy_order_price:
            return 1
        else:
            return 0


otcbtc = trade(driver)
for i in range(5):
    otcbtc.update_price(show=1)
    time.sleep(5)   

买一价 0.0010451
卖一价 0.00108999
现有差价 4.3 %


买一价 0.0010451
卖一价 0.00108999
现有差价 4.3 %


买一价 0.0010451
卖一价 0.00108999
现有差价 4.3 %


买一价 0.0010451
卖一价 0.00108999
现有差价 4.3 %


买一价 0.0010451
卖一价 0.00108999
现有差价 4.3 %


In [33]:
otcbtc.login_otcbtc()

请输入验证码


In [34]:
otcbtc.get_market_otceth()

In [46]:
otcbtc.buy_order()

TypeError: 'float' object is not callable

In [35]:
if otcbtc.check_buy_order():
    print(u'取消订单')
    otcbtc.check_buy_order()

TypeError: '>' not supported between instances of 'float' and 'NoneType'

'https://otcbtc.com/'